In [2]:
%pip install --upgrade --quiet  langchain langchain-community langchain-experimental langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
f = open('openaikey.txt')
os.environ["OPENAI_API_KEY"]=f.read()

In [20]:
# Examples of job roles and respective job titles
examples = [
    {"input": "software engineer", "output": "software development"},
    {"input": "accountant", "output": "accounting"},
    {"input": "teacher", "output": "education"},
    {"input": "doctor", "output": "medicine"},
    {"input": "architect", "output": "architecture"},
    {"input": "lawyer", "output": "law"},
]
# This is the list of examples available to select from.

In [26]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings,ChatOpenAI

chatAI = ChatOpenAI()

# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning
selector = SemanticSimilarityExampleSelector.from_examples(
    examples, # list of examples available to select from.
    OpenAIEmbeddings(), # embedding class used to produce embeddings which are used to measure semantic similarity.
    FAISS, # VectorStore class that is used to store the embeddings and do a similarity search over.
    k=5  # number of examples to produce.
)


In [31]:
from langchain_core.prompts import FewShotPromptTemplate
example_prompt = PromptTemplate.from_template("User input: {input}\nOutput: {output}")
similar_prompt = FewShotPromptTemplate(example_selector=selector, # The object that will help select examples
									   example_prompt=example_prompt,  # Your prompt
    								   prefix="Give the job title their job role is ", # Customizations that will be added to the top and bottom of your prompt
    								   suffix="Input: {job_title}\nOutput:",
    								   input_variables=["job_title"] # What inputs your prompt will receive
    								   )


In [32]:
prompt = similar_prompt.format(job_title="Nurse")
print(prompt)

Give the job title their job role is 

User input: doctor
Output: medicine

User input: teacher
Output: education

User input: software engineer
Output: software development

User input: lawyer
Output: law

User input: accountant
Output: accounting

Input: Nurse
Output:


In [33]:
result = chatAI.invoke(prompt)

In [34]:
print(result)

content='Healthcare' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 69, 'total_tokens': 71}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-e8c43953-57a0-46e0-a6e9-0d526b9b791a-0' usage_metadata={'input_tokens': 69, 'output_tokens': 2, 'total_tokens': 71}
